In [1]:
import pandas as pd
import random
from pathlib import Path

In [89]:
# Create references
f_refs = Path("citation_data/references.csv")
f_articles = Path("citation_data/articles.csv")
f_authors = Path("citation_data/authors.csv")

In [96]:
def write_file(df, path, overwrite=False):
    if path.exists():
        if overwrite:
            pass
        else:
            print("File exists already")
            return
    df.to_csv(path)

In [91]:
n_articles = 25
n_authors = 15

In [97]:
if f_refs.exists():
    refs = pd.read_csv(f_refs, index_col="id")
else:
    refs = pd.DataFrame(columns=["source", "target"])
    refs.index.name = "id"

    articles = [i for i in range(1,n_articles+1)]
    
    citations = {}

    for source in articles:
        if source == n_articles:
            continue
        citations[source] = []
        number_of_cites = random.randint(0, n_articles-source)
        
        candidates = [i for i in range(source+1, n_articles+1)]
        random.shuffle(candidates)
        for i in range(number_of_cites):
            citations[source].append(candidates.pop())

    index = 0
    for i, (source, targets) in enumerate(citations.items()):
        for target in targets:
            refs.loc[index] = [f"doi_{source}", f"doi_{target}"]
            index += 1
            
    write_file(refs, f_refs)

In [100]:
if f_articles.exists():
    articles = pd.read_csv(f_articles, index_col="doi")
else:
    articles = pd.DataFrame(columns=["author"], index=[f"doi_{i}" for i in range(1, n_articles+1)])

    authors = [i for i in range(1, n_authors+1)]
    while len(authors)<n_articles:
        authors.append(random.randint(1, n_authors))
    random.shuffle(authors)

    for i in articles.index:
        articles.loc[i, "author"] = f"author_{authors.pop()}"
        
    articles = articles.merge(refs.groupby("target").count().rename(columns={"source":"ref_count"}), left_index=True, right_index=True)
    
    articles.index.name = "doi"
    write_file(articles.sort_index(), f_articles)

In [102]:
authors = articles.groupby("author").mean().rename(columns={"ref_count":"avg_ref_count"})
authors = authors.merge(articles.groupby("author").sum().rename(columns={"source":"total_refs"}), left_index=True, right_index=True)
authors

,avg_ref_count,ref_count
author,,
author_1,3.0,3
author_10,4.5,9
author_11,7.5,15
author_14,7.0,14
author_15,7.0,7
author_2,3.0,6
author_3,9.0,9
author_4,8.5,17
author_5,11.0,11


In [105]:
authors.sort_index().to_csv(f_authors)